In [ ]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()
sqlContext = pyspark.sql.SQLContext(sc)

In [ ]:
import os

In [ ]:
from sift.corpora import wikipedia, wikidata
from sift.models import text, links
Conll_path = "/data/conll2003/"
base_path = '/data/wikipedia/20151002/'

In [ ]:
corpus = wikipedia.WikipediaCorpus()(sc, base_path + 'dump')
redirects = wikipedia.WikipediaRedirects()(corpus)

docs = ConllPrepare(Conll_path, "train", redirects)

In [ ]:
docs.take(1)

In [ ]:
wikipedia_pfx = 'en.wikipedia.org/wiki/'

In [ ]:
ec_model = links\
    .EntityCounts(min_count=5, filter_target=wikipedia_pfx)\
    .build(docs)\
    .map(links.EntityCounts.format_item)

In [ ]:
enc_model = links\
    .EntityNameCounts(lowercase=True, filter_target=wikipedia_pfx)\
    .build(docs)\
    .filter(lambda (name, counts): sum(counts.itervalues()) > 1)\
    .map(links.EntityNameCounts.format_item)

In [ ]:
ec_model.take(1)

In [ ]:
from nel.model import data
from nel.model.store import file

In [ ]:
os.environ['NEL_DATASTORE_URI'] = 'file:///data0/nel/'

In [ ]:
# we can use model.toLocalIterator if models don't fit in memory

In [ ]:
data.ObjectStore\
    .Get('models:ecounts[wikipedia]')\
    .save_many(ec_model.collect())

In [ ]:
data.ObjectStore\
    .Get('models:necounts[wikipedia]')\
    .save_many(enc_model.collect())

In [ ]:
from nel.doc import Doc

In [ ]:
from nel.harness.format import from_sift

In [ ]:
from nel.process.pipeline import Pipeline
from nel.process.candidates import NameCounts
from nel.features.probability import EntityProbability, NameProbability

In [ ]:
candidate_generation = [
    NameCounts('wikipedia', 10)
]
feature_extraction = [
    EntityProbability('wikipedia'),
    NameProbability('wikipedia')
]

In [ ]:
training_pipeline = Pipeline(candidate_generation + feature_extraction)

In [ ]:
training_docs = [from_sift(doc) for doc in docs.takeSample(False, 100)]

In [ ]:
train = [training_pipeline(doc) for doc in training_docs]

In [ ]:
from nel.learn import ranking
from nel.features import meta
from nel.model import resolution
from nel.process import resolve

In [ ]:
ranker = ranking.TrainLinearRanker(name='ranker', features=[f.id for f in feature_extraction])(train)

In [ ]:
classifier_feature = meta.ClassifierScore(ranker)
linking = [
    classifier_feature,
    resolve.FeatureRankResolver(classifier_feature.id)
]

In [ ]:
linking_pipeline = Pipeline(candidate_generation + feature_extraction + linking)

In [ ]:
sample = [from_sift(doc) for doc in docs.takeSample(False, 10)]

In [ ]:
# clear existing links
for doc in sample:
    for chain in doc.chains:
        chain.resolution = None
        for mention in chain.mentions:
            mention.resolution = None

In [ ]:
linked_sample = [linking_pipeline(doc) for doc in sample]

In [ ]:
[d.id for d in linked_sample]

In [ ]:
sample[0].chains[0].resolution.id